In [1]:
import sys
sys.path.append('../../')
import os
import time
import numpy
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf
import input_test
import math
import numpy as np
from i3d import InceptionI3d
from utils import *
import scipy.io as sio
import glob
# Basic model parameters as external flags.
flags = tf.app.flags
tf.app.flags.DEFINE_string('f', '', 'kernel')
gpu_num = 1
flags.DEFINE_integer('batch_size', 1, 'Batch size.')
flags.DEFINE_integer('num_frame_per_clib', 64, 'Nummber of frames per clib')
flags.DEFINE_integer('crop_size', 224, 'Crop_size')
flags.DEFINE_integer('rgb_channels', 3, 'Channels for input')
flags.DEFINE_integer('classics', 8, 'The num of class')
FLAGS = flags.FLAGS

os.environ["CUDA_VISIBLE_DEVICES"]="1"
outfile = './feat/all5e/'
if not os.path.exists(outfile):
    os.makedirs(outfile)



# Get the sets of images and labels for training, validation, and
# Tell TensorFlow that the model will be built into the default Graph.
pre_model_save_dir = "./models/rgb_imagenet_10000_6_64_0.0001_decay"
if not os.path.exists(pre_model_save_dir):
    print('WARNING !!!!! check the path model')
    exit()
else:
    test_list_file = '../../list/ucf24_list/alllist.list'
    file = list(open(test_list_file, 'r'))
    num_test_videos = len(file)
    print("Number of test videos={}".format(num_test_videos))
    with tf.Graph().as_default():
        rgb_images_placeholder, _, labels_placeholder, is_training = placeholder_inputs(
                        FLAGS.batch_size * gpu_num,
                        FLAGS.num_frame_per_clib,
                        FLAGS.crop_size,
                        FLAGS.rgb_channels
                        )

        with tf.variable_scope('RGB'):
            logit, _ = InceptionI3d(
                                num_classes=FLAGS.classics,
                                spatial_squeeze=True,
                                final_endpoint='Mixed_5c',
                                name='inception_i3d'
                                )(rgb_images_placeholder, is_training)
        norm_score = tf.nn.softmax(logit)

        # Create a saver for writing training checkpoints.
        saver = tf.train.Saver()
        init = tf.global_variables_initializer()

        # Create a session for running Ops on the Graph.
        sess = tf.Session(
                        config=tf.ConfigProto(allow_soft_placement=True)
                        )
        sess.run(init)

    ckpt = tf.train.get_checkpoint_state(pre_model_save_dir)
    if ckpt and ckpt.model_checkpoint_path:
        print ("loading checkpoint %s,waiting......" % ckpt.model_checkpoint_path)
        saver.restore(sess, ckpt.model_checkpoint_path)
        print ("load complete!")

    all_steps = num_test_videos
    top1_list = []
    for step in xrange(3008,all_steps):
        start_time = time.time()
        s_index = 0
        predicts = []
        data = {}
        top1 = False
        data_new = np.zeros((1,1024))
        idk = 0
        
        tot_frame = np.array(glob.glob(file[step].split(' ')[0]+ "/i/*.jpg")).shape[0]
        
        while True:
            val_images, _, val_labels, s_index, is_end = input_test.read_clip_and_label(
                            filename=file[step],
                            batch_size=FLAGS.batch_size * gpu_num,
                            s_index=s_index,
                            num_frames_per_clip=FLAGS.num_frame_per_clib,
                            crop_size=FLAGS.crop_size,
                            )

            data_features = sess.run(logit,
                               feed_dict={
                                            rgb_images_placeholder: val_images,
                                            labels_placeholder: val_labels,
                                            is_training: False
                                            })

            #print("feat ================")
            #print(data_features.shape)
            #print("feat ================")

            temp = np.mean(data_features, axis=1, keepdims = True)
            temp = np.mean(temp, axis=2, keepdims = True)
            temp = np.mean(temp, axis=3, keepdims = True)

            tmp = np.reshape(temp,(1,1024))

            #print("feat ================")
            #tmp = data_features
            
            #print("feat ================")
            #tmp = np.expand_dims(tmp, axis = 0)
  #          print(tmp.shape)
            if idk == 0:
                data_new += tmp
            else:
                data_new = np.concatenate([data_new,tmp], axis= 0)
            
            labell = np.zeros((1,24))
            labell[0,int(val_labels)] = 1

            #print(labell)
            idk += 1 
            s_index = idk
            if idk >= tot_frame:
                print(data_new.shape)
                data['feat'] = data_new
                data['label'] = labell

                temp = file[step].split("/")
                dirfile = temp[5].split(" ")
                namefile = temp[6].split(" ")
                
                #print(namefile[0])
                print(step)
                
                datasave = outfile+namefile[0]+'.mat'
                print(datasave)
                sio.savemat(datasave, data)
                break

Number of test videos=3207
loading checkpoint ./models/rgb_imagenet_10000_6_64_0.0001_decay/i3d_ucf_model-9999,waiting......
INFO:tensorflow:Restoring parameters from ./models/rgb_imagenet_10000_6_64_0.0001_decay/i3d_ucf_model-9999
load complete!
(240, 1024)
3008
./feat/all5e/v_SoccerJuggling_g05_c05.mat
(240, 1024)
3009
./feat/all5e/v_SoccerJuggling_g05_c06.mat
(251, 1024)
3010
./feat/all5e/v_SoccerJuggling_g06_c01.mat
(135, 1024)
3011
./feat/all5e/v_SoccerJuggling_g06_c02.mat
(229, 1024)
3012
./feat/all5e/v_SoccerJuggling_g06_c03.mat
(92, 1024)
3013
./feat/all5e/v_SoccerJuggling_g06_c04.mat
(300, 1024)
3014
./feat/all5e/v_SoccerJuggling_g06_c05.mat
(301, 1024)
3015
./feat/all5e/v_SoccerJuggling_g07_c01.mat
(301, 1024)
3016
./feat/all5e/v_SoccerJuggling_g07_c02.mat
(301, 1024)
3017
./feat/all5e/v_SoccerJuggling_g07_c03.mat
(301, 1024)
3018
./feat/all5e/v_SoccerJuggling_g07_c04.mat
(360, 1024)
3019
./feat/all5e/v_SoccerJuggling_g07_c05.mat
(360, 1024)
3020
./feat/all5e/v_SoccerJuggling

(65, 1024)
3148
./feat/all5e/v_VolleyballSpiking_g04_c01.mat
(61, 1024)
3149
./feat/all5e/v_VolleyballSpiking_g04_c02.mat
(76, 1024)
3150
./feat/all5e/v_VolleyballSpiking_g04_c03.mat
(111, 1024)
3151
./feat/all5e/v_VolleyballSpiking_g04_c04.mat
(98, 1024)
3152
./feat/all5e/v_VolleyballSpiking_g04_c05.mat
(110, 1024)
3153
./feat/all5e/v_VolleyballSpiking_g04_c06.mat
(129, 1024)
3154
./feat/all5e/v_VolleyballSpiking_g04_c07.mat
(104, 1024)
3155
./feat/all5e/v_VolleyballSpiking_g05_c01.mat
(66, 1024)
3156
./feat/all5e/v_VolleyballSpiking_g05_c02.mat
(59, 1024)
3157
./feat/all5e/v_VolleyballSpiking_g05_c03.mat
(63, 1024)
3158
./feat/all5e/v_VolleyballSpiking_g05_c04.mat
(94, 1024)
3159
./feat/all5e/v_VolleyballSpiking_g05_c05.mat
(121, 1024)
3160
./feat/all5e/v_VolleyballSpiking_g06_c01.mat
(106, 1024)
3161
./feat/all5e/v_VolleyballSpiking_g06_c02.mat
(121, 1024)
3162
./feat/all5e/v_VolleyballSpiking_g06_c03.mat
(148, 1024)
3163
./feat/all5e/v_VolleyballSpiking_g06_c04.mat
(53, 1024)
3164


In [ ]:
os.walk(filename_i)

In [ ]:
filename_i.filename

In [ ]:
tot_frame

In [ ]:
filename_i

In [ ]:
a = glob.glob(file[step].split(' ')[0]+ "/i/*.jpg")

In [ ]:
np.array(a).shape[0]